<a href="https://colab.research.google.com/github/byluca/llm-from-scratch/blob/main/llm_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### TOKENIZER USING BYTE PAIR ENCODING (BPE)

In [2]:
!pip3 install tiktoken

In [3]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Colab Notebooks/montecristo.txt'

Mounted at /content/drive


In [4]:
with open(file_path, 'r', encoding='utf-8') as f:
    raw_text = f.read()

print(f"Lunghezza del testo: {len(raw_text)} caratteri")

Lunghezza del testo: 2699512 caratteri


### IMPLEMENTING A DATA LOADER



In [5]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []    # lista per gli input
        self.target_ids = []   # lista per i target

        # Tokenizza tutto il testo
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Scorri il testo con una finestra mobile per creare sequenze sovrapposte
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]            # sequenza input
            target_chunk = token_ids[i + 1: i + max_length + 1]  # sequenza target spostata di 1
            self.input_ids.append(torch.tensor(input_chunk))     # aggiungi input come tensore
            self.target_ids.append(torch.tensor(target_chunk))   # aggiungi target come tensore

    def __len__(self):
        return len(self.input_ids)  # numero totale di sequenze

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]  # ritorna input e target per l’indice dato


In [6]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Inizializza il tokenizer GPT-2 per trasformare il testo in token
    tokenizer = tiktoken.get_encoding("gpt2")

    # Crea il dataset personalizzato usando il testo e il tokenizer
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Crea il DataLoader per gestire i dati in batch durante l'addestramento
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,   # numero di sequenze per batch
        shuffle=shuffle,         # mescola i dati se True
        drop_last=drop_last,     # scarta l'ultimo batch se incompleto
        num_workers=num_workers  # numero di processi per caricare i dati in parallelo
    )

    # Restituisce il DataLoader pronto all'uso
    return dataloader


<div class="alert alert-block alert-success">

Testiamo il dataloader con un batch size di 1 per un LLM con una dimensione del contesto di 4.

Questo aiuterà a sviluppare un’intuizione su come la classe GPTDatasetV1 e la funzione create_dataloader_v1 lavorano insieme:
</div>

In [8]:
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()


<div class="alert alert-block alert-info">

Converti il dataloader in un iteratore Python per ottenere la prossima voce usando la funzione built-in next() di Python.
</div>

In [12]:
import torch
import tiktoken

# Stampa la versione di PyTorch installata
print("PyTorch version:", torch.__version__)

# Crea un DataLoader dal testo raw_text con batch_size=1, sequenze di lunghezza 4, stride 1, senza mescolare i dati
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

# Trasforma il DataLoader in un iteratore Python per poter usare next()
data_iter = iter(dataloader)

# Prende il primo batch dall'iteratore
first_batch = next(data_iter)

# Stampa il primo batch (input e target)
print(first_batch)


PyTorch version: 2.6.0+cu124
[tensor([[  171,   119,   123, 33666]]), tensor([[  119,   123, 33666,  2579]])]


<div class="alert alert-block alert-warning">

La variabile first_batch contiene due tensori: il primo tensore memorizza gli ID dei token di input,
e il secondo tensore memorizza gli ID dei token target.

Dato che max_length è impostato a 4, ciascuno dei due tensori contiene 4 ID di token.

Nota che una dimensione di input pari a 4 è relativamente piccola ed è stata scelta solo a scopo illustrativo.
È comune addestrare LLM con dimensioni di input di almeno 256.
</div>

<div class="alert alert-block alert-success">

Per illustrare il significato di stride=1, prendiamo un altro batch da questo dataset:

</div>


In [13]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[  119,   123, 33666,  2579]]), tensor([[  123, 33666,  2579,  3158]])]


<div class="alert alert-block alert-warning">

Se confrontiamo il primo batch con il secondo, possiamo vedere che gli ID dei token del secondo batch sono spostati di una posizione rispetto al primo batch.

Per esempio, il secondo ID nell’input del primo batch è 123, che è il primo ID nell’input del secondo batch.

Il parametro stride determina di quante posizioni gli input si spostano da un batch all’altro, imitando un approccio a finestra mobile.
</div>

<div class="alert alert-block alert-warning">

I batch di dimensione 1, come quelli che abbiamo prelevato finora dal dataloader, sono utili a scopo illustrativo.

Se hai esperienza precedente con il deep learning, saprai che batch di piccole dimensioni richiedono meno memoria durante l’addestramento, ma portano a aggiornamenti del modello più rumorosi.

Proprio come nel deep learning tradizionale, la dimensione del batch è un compromesso e un iperparametro da sperimentare durante l’addestramento degli LLM.
</div>

<div class="alert alert-block alert-success">

Prima di passare alle due sezioni finali di questo capitolo, che si concentrano sulla creazione dei vettori di embedding dagli ID dei token, diamo una breve occhiata a come possiamo usare il dataloader per campionare con un batch size maggiore di 1:
</div>

In [14]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  171,   119,   123, 33666],
        [ 2579,  3158, 16057,   952],
        [ 1248,  1314,  8591,   410],
        [  276, 15253,   390,  8466],
        [42345,   390,  8466,  4860],
        [  544,  1062,   660,  4229],
        [  198,   325,  4593,  1000],
        [  390,  8466,   299,  1015]])

Targets:
 tensor([[  119,   123, 33666,  2579],
        [ 3158, 16057,   952,  1248],
        [ 1314,  8591,   410,   276],
        [15253,   390,  8466, 42345],
        [  390,  8466,  4860,   544],
        [ 1062,   660,  4229,   198],
        [  325,  4593,  1000,   390],
        [ 8466,   299,  1015,   257]])


<div class="alert alert-block alert-info">

Nota che aumentiamo lo stride a 4. Questo serve per utilizzare completamente il dataset (non saltiamo nessuna parola) ma anche per evitare sovrapposizioni tra i batch, poiché un’eccessiva sovrapposizione potrebbe portare a un aumento dell’overfitting.
</div>